In [1]:
git_token ="git token here"

In [5]:
from github import Github
import re
from datasets import Dataset

In [6]:
# initialize PyGithub with the GitHub token
g = Github(git_token)

In [7]:
# specify the repository
repo = g.get_repo("openai/gym")

In [8]:
# function to extract Python functions from a script
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    functions = pattern.findall(code)
    return functions

In [9]:
# fetch Python files from the repository
python_files = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    elif file_content.path.endswith(".py"):
        python_files.append(file_content)

# extract functions and create dataset
data = {"code": [], "function_name": []}
for file in python_files:
    code = file.decoded_content.decode("utf-8")
    functions = extract_functions_from_code(code)
    for function in functions:
        data["code"].append(code)
        data["function_name"].append(function)

# create a Hugging Face dataset
dataset = Dataset.from_dict(data)

# save the dataset to disk
dataset.save_to_disk("code_generation_dataset")

print("Dataset created and saved to disk.")

Saving the dataset (0/1 shards):   0%|          | 0/974 [00:00<?, ? examples/s]

Dataset created and saved to disk.


In the above code, we are initializing a GitHub client with a personal access token, specifying the “openai/gym” repository, and defining a function to extract Python function definitions from the code. We are then iterating over the contents of the repository to collect Python files, extracting function definitions from each file, and storing them in a dataset. In the end, we are creating a Hugging Face dataset from the extracted data and saving it to disk, which will allow us to use this dataset for tasks such as training or fine-tuning a code generation model.


Now, we will use a pre-trained LLM model from Salesforce to fine-tune the model on our dataset for the task of code generation:

In [10]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [11]:
# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

# set the pad_token to eos_token or add a new pad token
tokenizer.pad_token = tokenizer.eos_token

# load the dataset
dataset = load_from_disk("code_generation_dataset")

# split the dataset into training and test sets
dataset = dataset.train_test_split(test_size=0.1)

# preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['code'], truncation=True, padding='max_length')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

In the above code, we are initializing the tokenizer and model for code generation using a pre-trained model from Salesforce, and setting the pad token to ensure proper input formatting. We are then loading our previously saved dataset of code snippets from disk, splitting it into training and test sets to create a validation framework, and defining a preprocessing function to tokenize the code examples to ensure they are appropriately truncated and padded to a consistent length for model training.

The above step prepares the data for efficient fine-tuning of the code generation model. And now, here’s how to fine-tune the model:

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

trainer.train()

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In the above code, we are tokenizing the dataset by applying the preprocessing function in batches, which prepares the data for training. We then defined the training arguments, specifying parameters such as the output directory, batch size, number of training epochs, and checkpoint saving strategy. With these settings, we initialized a Trainer object with the model, training arguments, and the tokenized training and evaluation datasets. Finally, we started the fine-tuning process by calling the train method on the Trainer object, which begins training the model on the prepared dataset to improve its performance for code generation tasks.

This step will take time, depending on the computing power of your system. After this step, here’s how we can test our code generation model:

In [ ]:
# define a function to generate code using the fine-tuned model
def generate_code(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=max_length)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# test the model with a code generation prompt
prompt = "def merge_sort(arr):"
generated_code = generate_code(prompt)

print("Generated Code:")
print(generated_code)

Generated Code:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    while left and right: